<a href="https://colab.research.google.com/github/FeliGR/Madrid-Mortality-Forecast-Climate-Time-Series/blob/main/madrid_mortality_forecast_with_climate_exogenous_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis y Pronóstico de Mortalidad en Madrid usando Variables Climáticas: Enfoque de Series Temporales

## Integrantes del Proyecto
- **Felipe Guzmán**
- **Álvaro Alcalde**
- **Pablo Díaz-Masa**

## Curso
Este proyecto es realizado como parte del curso de **Aprendizaje Automático en Series Temporales y Flujos de Datos**.

## Descripción del Proyecto
El objetivo de este proyecto es realizar un análisis y pronóstico basado en datos históricos de mortalidad diaria en Madrid desde el año 2015. Nuestra variable objetivo es el número de defunciones corregidas registradas diariamente, y para enriquecer el análisis, hemos decidido incorporar variables exógenas relacionadas con el clima. Específicamente, utilizaremos las siguientes variables exógenas:
- **Temperatura media diaria**
- **Temperatura mínima diaria**
- **Temperatura máxima diaria**

Estas variables climáticas pueden tener un impacto directo sobre los patrones de mortalidad, por lo que su inclusión nos permitirá mejorar la precisión de nuestros modelos de pronóstico.

## Objetivos
- Analizar la serie temporal de mortalidad diaria en Madrid.
- Identificar tendencias, estacionalidades y patrones en los datos.
- Incorporar variables exógenas para enriquecer el modelo predictivo.
- Desarrollar un modelo de aprendizaje automático que permita predecir el número de defunciones corregidas utilizando técnicas específicas de series temporales.

## Fecha de Entrega
El proyecto tiene como fecha límite el **7 de octubre**, con una presentación preliminar entre el **1 y 3 de octubre**.


In [ ]:
!pip install skforecast

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

import warnings

In [ ]:
# Cargar los datos del archivo CSV a un DataFrame de pandas
mortalidad_madrid_df = pd.read_csv("mortalidad_madrid.csv")
# Imprimir las dimensiones del DataFrame (número de filas y columnas)
print(mortalidad_madrid_df.shape)

In [ ]:
# Contar los valores nulos en cada columna para detectar posibles datos faltantes
print("Número de valores Null:\n",mortalidad_madrid_df.isnull().sum())

In [ ]:
# Convertir la columna 'fecha' al tipo de dato datetime para facilitar el manejo de fechas
mortalidad_madrid_df['fecha'] = pd.to_datetime(mortalidad_madrid_df['fecha'], format='%Y-%m-%d')


In [ ]:
mortalidad_madrid_df.rename(columns={'defunciones_corregidas': 'defunciones'}, inplace=True)

In [ ]:
# Establecer la columna 'fecha' como índice del DataFrame para facilitar operaciones temporales
mortalidad_madrid_df.set_index('fecha', inplace=True)
# Revisar el índice para verificar que 'fecha' se ha establecido correctamente como índice
mortalidad_madrid_df.index

# Asegurarse de que el índice tenga una frecuencia definida
mortalidad_madrid_df = mortalidad_madrid_df.asfreq('D')

In [ ]:
# Graficar la serie temporal de defunciones corregidas para visualizar tendencias
mortalidad_madrid_df.head(5)

In [ ]:
# Verificar que un índice temporal está completo
# ==============================================================================
(mortalidad_madrid_df.index == pd.date_range(
                    start = mortalidad_madrid_df.index.min(),
                    end   = mortalidad_madrid_df.index.max(),
                    freq  = mortalidad_madrid_df.index.freq)
).all()

In [ ]:
# Definir el tamaño del conjunto de prueba como un 10% del total de datos
test_size_percentage = 0.1  # 10% de los datos totales

# Calcular el número de observaciones que corresponderán al conjunto de prueba
steps = int(len(mortalidad_madrid_df) * test_size_percentage)

# Generar los conjuntos de entrenamiento y prueba
mortalidad_madrid_train_df = mortalidad_madrid_df[:-steps]
mortalidad_madrid_test_df  = mortalidad_madrid_df[-steps:]

# Verificar las fechas de los conjuntos de entrenamiento y prueba
print(f"Fechas train : {mortalidad_madrid_train_df.index.min()} --- {mortalidad_madrid_train_df.index.max()}  (n={len(mortalidad_madrid_train_df)})")
print(f"Fechas test  : {mortalidad_madrid_test_df.index.min()} --- {mortalidad_madrid_test_df.index.max()}  (n={len(mortalidad_madrid_test_df)})")

# Graficar los datos
fig, ax = plt.subplots(figsize=(9, 4))
mortalidad_madrid_train_df['defunciones'].plot(ax=ax, label='train')
mortalidad_madrid_test_df['defunciones'].plot(ax=ax, label='test')
ax.legend()
plt.show()


### **EXPERIMENTO #1**

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = make_pipeline(StandardScaler(), RandomForestRegressor(random_state=123)),
                lags = 6
             )

forecaster.fit(y=mortalidad_madrid_train_df['defunciones'])

In [ ]:
test_size_percentage = 0.1  # 10% de los datos totales
# Calcular el número de observaciones que corresponderán al conjunto de prueba
steps = int(len(mortalidad_madrid_df) * test_size_percentage)

predictions = forecaster.predict(steps=steps)
predictions.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
mortalidad_madrid_train_df['defunciones'].plot(ax=ax, label='train')
mortalidad_madrid_test_df['defunciones'].plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predicciones')
ax.legend()
plt.show()

In [ ]:
error_mse = mean_absolute_error(
                y_true = mortalidad_madrid_test_df['defunciones'],
                y_pred = predictions
            )

print(f"Error de test (MAE) {error_mse}")

### **EXPERIMENTO #2**

In [ ]:
test_size_percentage = 0.1  # Ajusta este porcentaje según sea necesario
steps = int(len(mortalidad_madrid_df) * test_size_percentage)

forecaster = ForecasterAutoreg(
                regressor = make_pipeline(StandardScaler(), RandomForestRegressor(random_state=123)),
                lags      = 12 # Este valor será remplazado en el grid search
             )

# Lags utilizados como predictores
lags_grid = [10, 20]

# Hiperparámetros del regresor
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

param_grid = {'randomforestregressor__n_estimators': [100, 500, 1000],
              'randomforestregressor__max_depth': [3, 5, 10]}

resultados_grid = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = mortalidad_madrid_train_df['defunciones'],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = steps,
                        refit              = True,
                        metric             = 'mean_squared_error',
                        initial_train_size = int(len(mortalidad_madrid_train_df)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
                   )


In [ ]:
predictions = forecaster.predict(steps=steps)
predictions.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
mortalidad_madrid_train_df['defunciones'].plot(ax=ax, label='train')
mortalidad_madrid_test_df['defunciones'].plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predicciones')
ax.legend()
plt.show()

In [ ]:
error_mse = mean_absolute_error(
                y_true = mortalidad_madrid_test_df['defunciones'],
                y_pred = predictions
            )

print(f"Error de test (MAE): {error_mse}")

In [ ]:
# Crear y entrenar forecaster con mejores hiperparámetros
# ==============================================================================
regressor = RandomForestRegressor(max_depth=3, n_estimators=500, random_state=123)
forecaster = ForecasterAutoreg(
                regressor = regressor,
                lags      = 20
             )

forecaster.fit(y=mortalidad_madrid_train_df['defunciones'])

In [ ]:
predictions = forecaster.predict(steps=steps)
predictions.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
mortalidad_madrid_train_df['defunciones'].plot(ax=ax, label='train')
mortalidad_madrid_test_df['defunciones'].plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predicciones')
ax.legend()
plt.show()

In [ ]:
error_mse = mean_absolute_error(
                y_true = mortalidad_madrid_test_df['defunciones'],
                y_pred = predictions
            )

print(f"Error de test (MAE): {error_mse}")

### **EXPERIMENTO #3**

In [ ]:
# Ajustar el número de pasos (steps) si crees que ciclos más largos pueden ser relevantes
steps = 48  # Puedes probar con diferentes valores si hay ciclos más largos en los datos
n_backtesting = steps * 3  # Ajustar para un período más largo de backtesting

# Realizar el backtesting
metrica, predicciones_backtest = backtesting_forecaster(
                                    forecaster         = forecaster,
                                    y                  = mortalidad_madrid_train_df['defunciones'],
                                    initial_train_size = len(mortalidad_madrid_train_df) - n_backtesting,
                                    fixed_train_size   = True,  # Fijar el tamaño del conjunto de entrenamiento
                                    steps              = steps,
                                    refit              = True,  # Ajustar el modelo con cada nuevo conjunto de entrenamiento
                                    metric             = 'mean_squared_error',
                                    verbose            = True
                                 )

# Calcular métricas adicionales
metrica_rmse = np.sqrt(metrica)  # Raíz del MSE (RMSE)
metrica_mae = mean_absolute_error(y_true=mortalidad_madrid_train_df[-n_backtesting:]['defunciones'],
                                  y_pred=predicciones_backtest)

print(f"Error de backtest (MSE): {metrica}")
print(f"Error de backtest (RMSE): {metrica_rmse}")
print(f"Error de backtest (MAE): {metrica_mae}")



In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
mortalidad_madrid_train_df.loc[predicciones_backtest.index, 'defunciones'].plot(ax=ax, label='test')
predicciones_backtest.plot(ax=ax, label='predicciones')
ax.legend()
plt.show()